In [1]:
import json
import numpy as np
import matplotlib as plt

In [20]:
import json
import nltk
from difflib import SequenceMatcher

def calculate_similarity_score(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def evaluate_answers(abc_answer, xyz_answer):
    scores = {
        "Grammar": 0.0,
        "Coherence": 0.0,
        "Context": 0.0,
        "Correctness": 0.0,
    }

    # Grammar Score (based on grammatical correctness)
    scores["Grammar"] = 1.0 if nltk.edit_distance(abc_answer['answer'], xyz_answer['answer']) == 0 else 0.0

    # Coherence Score (based on logical consistency)
    scores["Coherence"] = calculate_similarity_score(abc_answer['answer'], xyz_answer['answer'])

    # Context Score (based on relevance to the provided context)
    context_score = calculate_similarity_score(abc_answer['context'], xyz_answer['context'])
    scores["Context"] = context_score

    # Correctness Score (based on factual accuracy)
    correctness_score = calculate_similarity_score(abc_answer['answer'], xyz_answer['answer'])
    scores["Correctness"] = correctness_score

    return scores

# Load data from JSON files
with open("/Users/vasu/Desktop/NLP /project/chatbot_ic_NLP/lib/backend/evaluations/generated_gpt3.5_context_and_answers.json", "r") as cellio_file:
    cellio = json.load(cellio_file)

with open("/Users/vasu/Desktop/NLP /project/chatbot_ic_NLP/lib/backend/evaluations/generated_gpt-4_context_and_answers.json", "r") as gpt4_file:
    gpt4 = json.load(gpt4_file)

# Assuming both files have answers to the same question with the same ID
question_id = cellio[0]['id']

# Find answers for the same question ID
celio_answer = next(item for item in cellio if item["id"] == question_id)
gpt4_answer = next(item for item in gpt4 if item["id"] == question_id)

# Evaluate answers
# Evaluate answers for Celio model
evaluation_scores_celio = evaluate_answers(celio_answer, gpt4_answer)

# Evaluate answers for GPT-4 model
evaluation_scores_gpt4 = evaluate_answers(gpt4_answer, celio_answer)

# Print evaluation scores for Celio model
print("Evaluation Scores for Celio Model:")
for criterion, score in evaluation_scores_celio.items():
    print(f"{criterion}: {score}")

# Print evaluation scores for GPT-4 model
print("\nEvaluation Scores for GPT-4 Model:")
for criterion, score in evaluation_scores_gpt4.items():
    print(f"{criterion}: {score}")




Evaluation Scores for Celio Model:
Grammar: 0.0
Coherence: 0.032507739938080496
Context: 0.5954105359457192
Correctness: 0.032507739938080496

Evaluation Scores for GPT-4 Model:
Grammar: 0.0
Coherence: 0.009287925696594427
Context: 0.5954105359457192
Correctness: 0.009287925696594427


In [24]:
# model_id = "healio"
# model_id = "gpt-3.5-turbo"
for model_id in ["gpt3.5", "gpt-4"]:
    with open(f"generated_{model_id}_context_and_answers.json", "r") as f:
        evaluations = json.load(f)
    scores = {
        "Grammar": 0.0,
        "Coherence": 0.0,
        "Context": 0.0,
        "Correctness": 0.0,
    }
    for q in evaluations:
        e = q["answer"]
        if "/10" in e:
            e = e.strip().replace("\n", "").replace("/10", "/10 ").replace(".", " ").replace(":", " ").replace("(", " ").replace(")", " ").split(" ")
            e = [w for w in e if w != ""]
            for i, tok in enumerate(e):
                if "/10" in tok:
                    try:
                        nom = float(tok.split("/")[0])
                        denom = float(tok.split("/")[1])
                        scores[e[i-1]] += nom / denom
                    except Exception as ex:
                        continue  # unnecessary total grades
        else:
            pass
            # print("GR
            # ADING FAILED!")
            # for k, v in q.items():
            #     print(k, v)
    print(model_id)
    for k, v in scores.items():
        print(f"    {k} : {np.round(v/6, 2)}/10")
    print("")

gpt3.5
    Grammar : 0.0/10
    Coherence : 0.0/10
    Context : 0.0/10
    Correctness : 0.0/10

gpt-4
    Grammar : 0.0/10
    Coherence : 0.0/10
    Context : 0.0/10
    Correctness : 0.0/10



In [25]:
import json
import nltk
from difflib import SequenceMatcher

def calculate_similarity_score(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def evaluate_answers(answers1, answers2):
    scores = {
        "Grammar": 0.0,
        "Coherence": 0.0,
        "Context": 0.0,
        "Correctness": 0.0,
    }

    # Calculate Grammar Score (based on grammatical correctness)
    grammar_score = sum(1 for answer1, answer2 in zip(answers1, answers2) if nltk.edit_distance(answer1['answer'], answer2['answer']) == 0)
    scores["Grammar"] = grammar_score / len(answers1)

    # Calculate Coherence Score (based on logical consistency)
    coherence_score = sum(calculate_similarity_score(answer1['answer'], answer2['answer']) for answer1, answer2 in zip(answers1, answers2)) / len(answers1)
    scores["Coherence"] = coherence_score

    # Calculate Context Score (based on relevance to the provided context)
    context_score = sum(calculate_similarity_score(answer1['context'], answer2['context']) for answer1, answer2 in zip(answers1, answers2)) / len(answers1)
    scores["Context"] = context_score

    # Calculate Correctness Score (based on factual accuracy)
    correctness_score = sum(calculate_similarity_score(answer1['answer'], answer2['answer']) for answer1, answer2 in zip(answers1, answers2)) / len(answers1)
    scores["Correctness"] = correctness_score

    return scores

# Load data from JSON files
with open("generated_gpt3.5_context_and_answers.json", "r") as f:
    answers_gpt3_5 = json.load(f)

with open("generated_gpt-4_context_and_answers.json", "r") as f:
    answers_gpt4 = json.load(f)

# Evaluate answers
evaluation_scores = evaluate_answers(answers_gpt3_5, answers_gpt4)

# Print evaluation scores
print("Evaluation Scores:")
for criterion, score in evaluation_scores.items():
    print(f"{criterion}: {score}")


Evaluation Scores:
Grammar: 0.0
Coherence: 0.054168465815008
Context: 0.37148701603879514
Correctness: 0.054168465815008
